In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from arch import arch_model

# Download data
start_date = '2005-01-01'
end_date = '2025-01-01'

# Get S&P 500 data
sp500 = yf.download('^GSPC', start=start_date, end=end_date, auto_adjust=True)
sp500 = sp500[['Close']].copy()
sp500.columns = ['sp500_close']
sp500['sp500_log_ret'] = np.log(sp500['sp500_close'] / sp500['sp500_close'].shift(1))

# Get VIX data
vix = yf.download('^VIX', start=start_date, end=end_date, auto_adjust=True)
vix = vix[['Close']].copy()
vix.columns = ['vix_close']
vix['vix_log_ret'] = np.log(vix['vix_close'] / vix['vix_close'].shift(1))

# Merge datasets on date index
data = pd.merge(sp500, vix, left_index=True, right_index=True, how='inner')
data.dropna(inplace=True)  # Remove first row with NaN returns
print(f"Data shape: {data.shape}")
print(data.head())

# Fit GARCH model on S&P 500 log returns
garch_model = arch_model(
    data['sp500_log_ret'], 
    mean='Constant',  # Important for proper residual calculation
    vol='GARCH', 
    p=1, 
    q=1
)
garch_result = garch_model.fit(update_freq=0, disp='off')
print(garch_result.summary())

# Add standardized residuals to dataframe
data['garch_resid'] = garch_result.std_resid

# Train-test split (60-40)
split_idx = int(0.6 * len(data))
train_data = data.iloc[:split_idx]
test_data = data.iloc[split_idx:]

# Scale VIX log returns using training data
vix_scaler = StandardScaler()
train_data['vix_log_ret_scaled'] = vix_scaler.fit_transform(train_data[['vix_log_ret']])
test_data['vix_log_ret_scaled'] = vix_scaler.transform(test_data[['vix_log_ret']])

# Sequence creation
SEQ_LEN = 30

def create_sequences(df):
    xs, ys = [], []
    for i in range(SEQ_LEN, len(df)):
        # Input: [GARCH residual, Scaled VIX return]
        x = df[['garch_resid', 'vix_log_ret_scaled']].iloc[i-SEQ_LEN:i].values
        # Target: Next period's scaled VIX return
        y = df['vix_log_ret_scaled'].iloc[i]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

X_train, y_train = create_sequences(train_data)
X_test, y_test = create_sequences(test_data)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# PyTorch Dataset
class FinanceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_ds = FinanceDataset(X_train, y_train)
test_ds = FinanceDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

# LSTM Model (same as before)
class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1]  # Last timestep
        return self.linear(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
EPOCHS = 50
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch+1) % 10 == 0:
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.6f}')

# Evaluation and price conversion
model.eval()
with torch.no_grad():
    # Predict scaled log returns
    test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    preds_scaled = model(test_tensor).cpu().numpy().squeeze()
    
    # Inverse scaling to actual log returns
    pred_log_ret = vix_scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
    true_log_ret = test_data['vix_log_ret'].iloc[SEQ_LEN:].values
    
    # Convert log returns to prices
    # Get previous closing prices for conversion
    prev_prices = test_data['vix_close'].iloc[SEQ_LEN-1:-1].values
    pred_prices = prev_prices * np.exp(pred_log_ret)
    true_prices = test_data['vix_close'].iloc[SEQ_LEN:].values
    
    # Calculate metrics
    logret_r2 = r2_score(true_log_ret, pred_log_ret)
    logret_rmse = np.sqrt(mean_squared_error(true_log_ret, pred_log_ret))
    price_rmse = np.sqrt(mean_squared_error(true_prices, pred_prices))

print("\nLog Return Metrics:")
print(f"R²: {logret_r2:.4f}")
print(f"RMSE: {logret_rmse:.6f}")

print("\nPrice Metrics:")
print(f"Price RMSE: {price_rmse:.4f}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
c:\Users\danch\anaconda3\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001465. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
C:\Users\danch\AppData\Local\Temp\ipykernel_22192\38965785.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['vix_log_ret_scaled'] = vix_scale

Data shape: (5032, 4)
            sp500_close  sp500_log_ret  vix_close  vix_log_ret
Date                                                          
2005-01-04  1188.050049      -0.011740      13.98    -0.007128
2005-01-05  1183.739990      -0.003634      14.09     0.007838
2005-01-06  1187.890015       0.003500      13.58    -0.036867
2005-01-07  1186.189941      -0.001432      13.49    -0.006649
2005-01-10  1190.250000       0.003417      13.23    -0.019462
                     Constant Mean - GARCH Model Results                      
Dep. Variable:          sp500_log_ret   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                5067.30
Distribution:                  Normal   AIC:                          -10126.6
Method:            Maximum Likelihood   BIC:                          -10100.5
                                        No. Observations:      